In [1]:
# let's create a script to extract data from : https://www.droitsquotidiens.be/fr/service-juriweb

In [2]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re

In [3]:
def extract_text_between_keywords(text, keyword1, keyword2):
    try:
        start_index = text.index(keyword1) + len(keyword1)
        end_index = text.index(keyword2)
        return text[start_index:end_index].strip()
    except ValueError:
        return "Keywords not found in the text."


def extract_text_before_keyword(text, keyword):
    try:
        end_index = text.index(keyword)
        extracted_text = text[:end_index].strip()

        # Replace multiple spaces with a single space
        cleaned_text = re.sub(r"\s+", " ", extracted_text)

        return cleaned_text
    except ValueError:
        return "Keyword not found in the text."


def extract_QR(url):
    url = "https://www.droitsquotidiens.be" + url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    raw_text = soup.get_text()

    raw_text = raw_text.lower()
    raw_text = raw_text.replace("\n", " ")

    raw_text = re.sub(r"\s+", " ", raw_text)

    # Example usage
    keyword1 = "vous pas encore abonné ?"
    keyword2 = "envoyer par e-mail"

    response = extract_text_between_keywords(raw_text, keyword1, keyword2)
    keyword = "droits quotidiens -"
    question = extract_text_before_keyword(raw_text, keyword)

    return (question, response)

In [4]:
def extract_links_with_keyword(url, keyword):
    try:
        # Fetch the content of the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract all anchor tags
    anchor_tags = soup.find_all("a")

    # Extract href attributes that contain the keyword
    links = [
        tag.get("href")
        for tag in anchor_tags
        if tag.get("href") and keyword in tag.get("href")
    ]

    return links


# Example usage
url = "https://www.droitsquotidiens.be/fr/categories/famille"
keyword = "question"
links = extract_links_with_keyword(url, keyword)

In [6]:
y = []
for url in links:
    x = extract_QR(url)
    y.append(x)

In [12]:
import pandas as pd

dat = pd.DataFrame(y).rename(columns={0: "question", 1: "answer"})

In [13]:
dat.to_csv("droit_quotidien_QR.csv")